In [ ]:
# Useful starting lines
%matplotlib inline

import numpy as np
import scipy
import scipy.io
import scipy.sparse as sp
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the Data
Note that `ratings` is a sparse matrix that in the shape of (num_items, num_users)

In [ ]:
from helpers import load_data, preprocess_data

path_dataset = "../data/data_train.csv"
ratings = load_data(path_dataset)

### Plot the number of ratings per movie and user

In [ ]:
from plots import plot_raw_data

num_items_per_user, num_users_per_item = plot_raw_data(ratings)

print("min # of items per user = {}, min # of users per item = {}.".format(
        min(num_items_per_user), min(num_users_per_item)))

### Split the data into a train and test set

In [ ]:
def split_data(ratings, num_items_per_user, num_users_per_item,
               min_num_ratings, p_test=0.1):
    """split the ratings to training data and test data.
    Args:
        min_num_ratings: 
            all users and items we keep must have at least min_num_ratings per user and per item. 
    """
    # set seed
    np.random.seed(998)
    
    # select user and item based on the condition.
    valid_users = np.where(num_items_per_user >= min_num_ratings)[0]
    valid_items = np.where(num_users_per_item >= min_num_ratings)[0]
    valid_ratings = ratings[valid_items, :][: , valid_users]  
    
    xs, ys = valid_ratings.nonzero()
    indices = list(zip(xs, ys))
    np.random.shuffle(indices)
    
    cut = int(p_test * len(indices))
    train = valid_ratings.copy()
    xs, ys = zip(*indices)
    train[xs[:cut], ys[:cut]] = 0
    test = valid_ratings.copy()
    test[xs[cut:], ys[cut:]] = 0
    
    print("Total number of nonzero elements in origial data:{v}".format(v=ratings.nnz))
    print("Total number of nonzero elements in train data:{v}".format(v=train.nnz))
    print("Total number of nonzero elements in test data:{v}".format(v=test.nnz))
    return valid_ratings, train, test

In [ ]:
from plots import plot_train_test_data

valid_ratings, train, test = split_data(
    ratings, num_items_per_user, num_users_per_item, min_num_ratings=10, p_test=0.1)
plot_train_test_data(train, test)

## Implementing Baselines 

### Use the global mean to do the prediction

In [ ]:
from helpers import calculate_mse

def baseline_global_mean(train, test):
    """baseline method: use the global mean."""
    avg = test.copy()
    avg[avg.nonzero()] = train.sum() / train.size 
    return np.sqrt((1/2) * calculate_mse(test, avg).sum() / avg.size)

#tr_rmse = baseline_global_mean(train, train)
te_rmse = baseline_global_mean(train, test)
#print("train RMSE : ", tr_rmse)
print("test RMSE : ", te_rmse)

### Use the user means as the prediction

In [ ]:
def baseline_user_mean(train, test):
    """baseline method: use the user means as the prediction."""    
    avg = test.copy()
    xs, ys = avg.nonzero()
    for i in np.arange(train.shape[0]):
        avg[i, ys[xs == i]] = train[i, :].sum() / train[i, :].size
    return np.sqrt((1/2) * calculate_mse(avg, test).sum() / avg.size)


#tr_rmse = baseline_user_mean(train, train)
te_rmse = baseline_user_mean(train, test)
#print("train RMSE : ", tr_rmse)
print("test RMSE : ", te_rmse)

### Use the item means as the prediction

In [ ]:
def baseline_item_mean(train, test):
    """baseline method: use item means as the prediction."""
    avg = test.copy()
    xs, ys = avg.nonzero()
    for u in np.arange(train.shape[1]):
        avg[xs[ys == u], u] = train[:, u].sum() / train[:, u].size    
    return np.sqrt(calculate_mse(avg, test).sum()  / avg.size)
    

#tr_rmse = baseline_item_mean(train, train)
te_rmse = baseline_item_mean(train, test)
#print("train RMSE : ", tr_rmse)
print("test RMSE : ", te_rmse)

In [ ]:
from helpers import predict

def predict_baseline(train, pred):
    avg = pred.copy()
    xs, ys = avg.nonzero()
    for u in np.arange(train.shape[1]):
        avg[xs[ys == u], u] = train[:, u].sum() / train[:, u].size
    predict(avg)
    
pred = load_data("../data/sampleSubmission.csv")
predict_baseline(train, pred)

### Learn the Matrix Factorization using SGD

#### Initialize matrix factorization

In [ ]:
def init_MF(train, num_features):
    """init the parameter for matrix factorization."""
    num_item, num_user = train.shape
    item_features = np.random.random((num_features, num_item)) * 5 # W
    user_features = np.random.random((num_features, num_user)) * 5 # Z
    return user_features, item_features

Compute the cost by the method of matrix factorization.


In [ ]:
def compute_error(data, user_features, item_features, nz):
    """compute the loss (MSE) of the prediction of nonzero elements."""
    return np.sum(np.square(data - item_features.T @ user_features) / 2) / nz

In [ ]:
user_features, item_features = init_MF(train, 20)
compute_error(test, user_features, item_features, test.size)

In [ ]:
def matrix_factorization_SGD(train, test):
    """matrix factorization by SGD."""
    # define parameters
    gamma = 0.01
    num_features = 20   # K in the lecture notes
    lambda_user = 0.1
    lambda_item = 0.7
    num_epochs = 20     # number of full passes through the train set
    errors = [0]
    
    # set seed
    np.random.seed(988)

    # init matrix
    user_features, item_features = init_MF(train, num_features)
    
    # find the non-zero ratings indices 
    nz_row, nz_col = train.nonzero()
    nz_train = list(zip(nz_row, nz_col))
    nz_row, nz_col = test.nonzero()
    nz_test = list(zip(nz_row, nz_col))
    
    print("learn the matrix factorization using SGD...")
    for it in range(num_epochs):        
        # shuffle the training rating indices
        np.random.shuffle(nz_train)
        
        # decrease step size
        gamma /= 1.2
        
        for d, n in nz_train:
            e = train[d, n] - user_features[:, n].T @ item_features[:, d]
            
            dw = e * user_features[:, n]
            dz = e * item_features[:, d]
            item_features[:, d] += gamma * (dw - lambda_item * e) 
            user_features[:, n] += gamma * (dz - lambda_user * e)
            
            #item_features[:, d] += gamma * (dw - lambda_item * np.linalg.norm(item_features) * 0.001) 
            #user_features[:, n] += gamma * (dz - lambda_user * np.linalg.norm(user_features) * 0.001)
            
        rmse = compute_error(train, user_features, item_features, train.size)
        print("iter: {}, RMSE on training set: {}.".format(it, rmse))
        
        errors.append(rmse)
    
    rmse = compute_error(test, user_features, item_features, test.size)
    print("RMSE on test data: {}.".format(rmse))
    plt.plot(errors)
    plt.show()
    
matrix_factorization_SGD(train, test)   

### Learn the Matrix Factorization using Alternating Least Squares

In [ ]:
def update_user_feature(
        train, item_features, lambda_user,
        nnz_items_per_user, nz_user_itemindices):
    """update user feature matrix."""
    w = item_features
    return np.linalg.inv(w @ w.T + np.identity(w.shape[0]) * lambda_user) @ (w @ train) 

def update_item_feature(
        train, user_features, lambda_item,
        nnz_users_per_item, nz_item_userindices):
    """update item feature matrix."""
    z = user_features
    return np.linalg.inv(z @ z.T + np.identity(z.shape[0]) * lambda_item) @ (z @ train.T)

In [ ]:
from helpers import build_index_groups


def ALS(train, test):
    """Alternating Least Squares (ALS) algorithm."""
    # define parameters
    num_features = 20   # K in the lecture notes
    lambda_user = 0.1
    lambda_item = 0.7
    stop_criterion = 1e-4
    change = 1
    error_list = [0]
    
    # set seed
    np.random.seed(988)

    # init ALS
    user_features, item_features = init_MF(train, num_features)
    
    rmse = compute_error(train, user_features, item_features, train.size)

    while True:
        item_features = update_item_feature(train, user_features, lambda_item, 1, 1)
        user_features = update_user_feature(train, item_features, lambda_user, 1, 1)
        
        rmse = compute_error(train, user_features, item_features, train.size)
        error_list.append(rmse)
        print(rmse)
        if abs(error_list[-2] - error_list[-1]) < stop_criterion:
            break
        
    plt.plot(error_list)
    plt.show()

ALS(train, test)